<a href="https://colab.research.google.com/github/AimjGuytidy/Financial_engineering/blob/main/ML_devops.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install --upgrade google-cloud-bigquery

     |████████████████████████████████| 201 kB 28.5 MB/s 
     |████████████████████████████████| 42 kB 1.5 MB/s 
     |████████████████████████████████| 75 kB 4.3 MB/s 
     |████████████████████████████████| 92 kB 460 kB/s 
  Attempting uninstall: google-api-core
    Found existing installation: google-api-core 1.26.3
    Uninstalling google-api-core-1.26.3:
      Successfully uninstalled google-api-core-1.26.3
  Attempting uninstall: google-resumable-media
    Found existing installation: google-resumable-media 0.4.1
    Uninstalling google-resumable-media-0.4.1:
      Successfully uninstalled google-resumable-media-0.4.1
  Attempting uninstall: google-cloud-core
    Found existing installation: google-cloud-core 1.0.3
    Uninstalling google-cloud-core-1.0.3:
      Successfully uninstalled google-cloud-core-1.0.3
  Attempting uninstall: google-cloud-bigquery
    Found existing installation: google-cloud-bigquery 1.21.0
    Uninstalling google-cloud-bigquery-1.21.0:
      Successful

In [ ]:
# !set GOOGLE_APPLICATION_CREDENTIALS="C:\Users\user\Downloads\parfait-326913-0cd36d73431d.json"

In [ ]:
# !pip install --upgrade 'google-cloud-bigquery[bqstorage,pandas]'

In [ ]:
%load_ext google.cloud.bigquery

In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [ ]:
%%bigquery df --project parfait-326913 
SELECT
  mother_age,
  COUNT(1) AS num_babies,
  AVG(weight_pounds) AS avg_wt
FROM
  publicdata.samples.natality
WHERE
  year > 2000
GROUP BY mother_age
ORDER BY mother_age

Downloading: 100%|██████████| 45/45 [00:00<00:00, 180.99rows/s]


In [ ]:
df.head()

,mother_age,num_babies,avg_wt
0,10,3,7.533195
1,11,44,6.149945
2,12,1196,6.506330
3,13,7552,6.656063
4,14,44868,6.743684


In [ ]:
%%bigquery air --project parfait-326913 
SELECT 
  DISTINCT(departure_airport)
FROM bigquery-samples.airline_ontime_data.flights

Downloading: 100%|██████████| 347/347 [00:00<00:00, 1558.79rows/s]


In [ ]:
air.head()

,departure_airport
0,SNA
1,IAD
2,ORD
3,DTW
4,KOA


In [ ]:
import tensorflow as tf


In [ ]:
num_buckets = 1000
tf.feature_column.categorical_column_with_hash_bucket(air['departure_airport'][4],num_buckets,dtype=tf.dtypes.string)

HashedCategoricalColumn(key='KOA', hash_bucket_size=1000, dtype=tf.string)

In [ ]:
keywords = tf.feature_column.categorical_column_with_hash_bucket("keywords",
10000)
columns = [keywords]
features = {'keywords': tf.constant([['Tensorflow', 'Keras', 'RNN', 'LSTM',
'CNN'], ['LSTM', 'CNN', 'Tensorflow', 'Keras', 'RNN'], ['CNN', 'Tensorflow',
'LSTM', 'Keras', 'RNN']])}

In [ ]:
columns

[HashedCategoricalColumn(key='keywords', hash_bucket_size=10000, dtype=tf.string)]

In [ ]:
linear_prediction, _, _ = tf.compat.v1.feature_column.linear_model(features,
columns)

Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/base_layer.py:2233: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


In [ ]:
linear_prediction

<tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.], dtype=float32)>

In [ ]:
%%bigquery hashy --project parfait-326913
CREATE TEMPORARY FUNCTION hashed(airports STRING, numbuckets INT64) AS ( 
  ABS(MOD(FARM_FINGERPRINT(airports),numbuckets))
  );

WITH airports AS (
  SELECT 
  DISTINCT(departure_airport)
FROM bigquery-samples.airline_ontime_data.flights
)
SELECT 
  *,
  hashed(departure_airport,3) AS hash3,
  hashed(departure_airport,10) AS hash10,
  hashed(departure_airport,1000) AS hash1000
FROM airports

Downloading: 100%|██████████| 347/347 [00:00<00:00, 1450.88rows/s]


In [ ]:
hashy.head()

,departure_airport,hash3,hash10,hash1000
0,OAK,0,7,487
1,ITO,2,6,126
2,IND,2,2,692
3,ELP,2,3,193
4,GUC,0,2,942


In [ ]:
%%bigquery hasha --project parfait-326913
CREATE TEMPORARY FUNCTION hashed(airports STRING, numbuckets INT64) AS(
  ABS(MOD(FARM_FINGERPRINT(airports),numbuckets))
);
WITH airports AS (
  SELECT
    departure_airport,COUNT(1) AS num_flights
  FROM bigquery-samples.airline_ontime_data.flights
  GROUP BY departure_airport
)
SELECT
  departure_airport,num_flights
FROM airports
WHERE hashed(departure_airport,100) = hashed('ORD',100)

Downloading: 100%|██████████| 3/3 [00:00<00:00, 12.23rows/s]


In [ ]:
hasha

,departure_airport,num_flights
0,ORD,3610491
1,MCI,597761
2,BTV,66555


page 79